In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from IPython.display import display, Markdown
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

# Lettura Dataset

In [2]:
raw_dataset = pd.read_csv('../data/raw_dataset.csv')

# Conversione Testo (text-clening)

### Conversione in Stringa della Colonna 'text_'

Per prima cosa viene convertita in stringa la feature "text_" per ogni entry del Dataset. Questo tipo di operazione è utile ad assicurarsi che i dati nella colonna 'text_' siano trattati come testo. È utile anche perchè i dati potrebbero essere stati importati con tipi misti o non corretti.


In [4]:
raw_dataset['text_'] = raw_dataset['text_'].astype(str)

### Conversione in lowercase della Colonna 'text_'

Adesso, appurato che la colonna "text_" contiene solo stringhe, si va a effettuare un'operazione di lowercasing, dove ogni lettera contenuta nella colonna "text_" viene convertita in minucola. Questo è fondamentale in analisi del testo e machine learning, dove le differenze tra maiuscole e minuscole spesso non sono significative e possono creare rumore.

In [6]:
raw_dataset['text_'] = raw_dataset['text_'].str.lower()

# Pulizia del testo

**(Da controllare!)** Successivamente viene definita una routine di operazioni utile a migliorare la qualità del testo. Vediamo perché ciascuna di queste operazioni potrebbe essere utile in questo contesto:

**Tokenizzazione:** Suddividere il testo in parole o token è essenziale per analizzare la struttura e il contenuto delle recensioni. In analisi del testo, specialmente in contesti come il rilevamento delle recensioni false, è importante esaminare le parole individuali per identificare modelli, uso di linguaggio insolito, o altre caratteristiche che potrebbero indicare una recensione falsa.

**Rimozione delle Stop Words:** Le stop words sono generalmente parole comuni che non aggiungono molto significato al testo. Nel contesto del rilevamento di recensioni false, rimuovere queste parole può aiutare a ridurre il rumore nei dati e consentire al modello di concentrarsi su parole più significative che possono essere più indicative di recensioni autentiche o false.

**Rimozione dei Numeri:** I numeri spesso non contribuiscono al rilevamento delle recensioni false, a meno che non siano specificamente rilevanti per il contesto. Rimuovendoli, si riduce ulteriormente il rumore nei dati, permettendo al modello di concentrarsi sul testo chiave.

**Rimozione della Punteggiatura:** La punteggiatura può introdurre rumore e complicazioni nell'analisi del testo. Rimuovendo i segni di punteggiatura, si semplifica il testo per l'analisi. Tuttavia, in alcuni casi, la punteggiatura (come l'eccessivo uso di punti esclamativi) potrebbe essere un indicatore di recensioni false, quindi questa decisione dovrebbe essere basata sulla specifica natura del dataset e sull'obiettivo dell'analisi.

Queste operazioni di pre-elaborazione aiutano a pulire e standardizzare i dati del testo, migliorando la qualità delle informazioni che alimentano il modello di machine learning. Ciò è particolarmente importante nel rilevamento delle recensioni false, dove la precisione e la chiarezza del testo analizzato possono avere un impatto significativo sui risultati del modello.



In [7]:
def text_cleaning(text):
    return ' '.join([word for word in word_tokenize(text) 
                     if word not in stopwords.words('english') 
                     and not word.isdigit() 
                     and word not in string.punctuation])

### Test routine "text_cleaning"

Adesso stampiamo due volte la colonna "text_" delle prime 3 entry del Dataset. La prima volta stampiamo i valori di partenza, mentre la seconda volta andiamo ad applicare la routine "text_cleaning".

In [8]:
# Stampa le prime 3 stringhe senza pulizia
display(Markdown('**Stringhe senza pulizia testo:**'))
for i in range(3):
    string_dirty = raw_dataset['text_'][i]
    display(Markdown(f'{i+1}. {string_dirty}'))

# Stampa le prime 3 stringhe con pulizia
display(Markdown('**Stringhe con pulizia testo:**'))
for i in range(3):
    string_cleaned = text_cleaning(raw_dataset['text_'][i])
    display(Markdown(f'{i+1}. {string_cleaned}'))

**Stringhe senza pulizia testo:**

1. love this!  well made, sturdy, and very comfortable.  i love it!very pretty

2. love it, a great upgrade from the original.  i've had mine for a couple of years

3. this pillow saved my back. i love the look and feel of this pillow.

**Stringhe con pulizia testo:**

1. love well made sturdy comfortable love pretty

2. love great upgrade original 've mine couple years

3. pillow saved back love look feel pillow

Testata la corretta esecuzione della routine, verrà adesso applicata a tutte le entry (routine con tempi molto lunghi):

In [9]:
raw_dataset['text_'] = raw_dataset['text_'].apply(text_cleaning)